# Liberias 

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

In [ ]:
#ruta para los csv en data/processed
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Subir un nivel
DATA_DIR = os.path.join(BASE_DIR, "data", "processed") 

# Rand-Forest

## Feaure Importance
Medimos la importancia de cada variable para ver las que vamosa considerar en nuestro modelo


In [ ]:
def evaluar_importancia_variables(data_dir):
    resultados = {}

    # Lista de archivos esperados
    file_paths = {
        "v1": os.path.join(data_dir, "Propensity_clean_v1.csv"),
        "v2": os.path.join(data_dir, "Propensity_clean_v2.csv"),
        "v3": os.path.join(data_dir, "Propensity_clean_v3.csv"),
        "v4": os.path.join(data_dir, "Propensity_clean_v4.csv"),
    }

    for version, path in file_paths.items():
        print(f"\n🔍 Evaluando importancia de variables en: {version}")

        # Cargar los datos
        df = pd.read_csv(path)

        # Verificar si la columna "Mas_1_coche" está presente y eliminar columnas problemáticas
        if "Mas_1_coche" in df.columns:
            y = df["Mas_1_coche"]
            X = df.drop(columns=["Mas_1_coche"], errors="ignore")
        # Eliminar columnas no numéricas si existen
        X = X.select_dtypes(include=['number'])

        # Dividir en entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Entrenar un Random Forest básico
        rf = RandomForestClassifier(n_estimators=100, random_state=42)
        rf.fit(X_train, y_train)

        # Obtener importancia de variables
        importances = rf.feature_importances_
        feature_importance_df = pd.DataFrame({"Variable": X.columns, "Importancia": importances})
        feature_importance_df = feature_importance_df.sort_values(by="Importancia", ascending=False)

        # Guardar resultados
        resultados[version] = feature_importance_df

        # Mostrar la tabla de importancia de variables
        display(feature_importance_df)

        # Graficar la importancia de las variables
        plt.figure(figsize=(10, 6))
        sns.barplot(x="Importancia", y="Variable", data=feature_importance_df)
        plt.title(f"Importancia de Variables en Random Forest - {version}")
        plt.show()

    return resultados


# Ejecutar la función con la ruta corregida
importancia_variables_resultados = evaluar_importancia_variables(DATA_DIR)


Como en cada csv hemos hecho un tratamiento distinto de los nulos , comparamos la importancia de cada variable en cada uno de ellos

In [ ]:
comparacion_df = pd.DataFrame()

for version, df_importance in importancia_variables_resultados.items():
    df_importance = df_importance.rename(columns={"Importancia": f"Importancia_{version}"})
    if comparacion_df.empty:
        comparacion_df = df_importance
    else:
        comparacion_df = comparacion_df.merge(df_importance, on="Variable", how="outer")


comparacion_df["Importancia_Promedio"] = comparacion_df.drop(columns=["Variable"]).mean(axis=1)
comparacion_df = comparacion_df.sort_values(by="Importancia_Promedio", ascending=False)
comparacion_df = comparacion_df.drop(columns=["Importancia_Promedio"])

display(comparacion_df)


**Exclusión de la Variable `Tiempo`**

Tiempo porque es un parámetro que solo tienen los clientes con una segunda compra. Esto sesga la predicción, ya que no es representativo de todos los clientes, especialmente aquellos que no han realizado una segunda compra. 